<a href="https://colab.research.google.com/github/gagomezuribe/03MIAR--Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Gustavo_Gomez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Gustavo Adolfo Gómez Uribe  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
**>1. Sesiones de doblaje <br>**

**Descripción del problema:**
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

Número de actores: 10; Número de tomas : 30; Actores/Tomas Cuadro en : https://bit.ly/36D8IuK 1=Participa en la toma








                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

**Espacio de soluciones:**
Puede ser representado como un vector de 30 elementos, donde el índice corresponde a la toma y el valor al día en que se debe realizar dicha toma.

Se debe cumplir las restricciones del problema para que una solución sea válida:
1: el dia debe estar entre 1 y el máximo, es decir, tomas total/tomas diarias, para este caso, 30/6 = 5, día pertenece [1,5]

**Funcion objetivo:**
Se puede usar una matriz de apoyo de tantas columnas como actores y filas como tomas y el valor es el dia en que un actor hace la toma.

Luego se hace una matriz de la participación de los actores por día (para una o varias tomas) y el totalizado de estas columnas es la cantidad de actores por día, lo cual al sumarse por todos los dias sería la función objetivo a minimizar (modelo en hoja de cálculo)

https://docs.google.com/spreadsheets/d/124dato8bOGx0mvLeA90dbDtS0EiqvMm1zZ0rViwl2AY/edit?usp=sharing




**Implementación de las restricciones:**
Al crear una nueva solución debe validarse en el código que no existan mas de 6 tomas por día, es decir, que ningún día se repita más de 5 veces en la solución a analizar

In [ ]:
#Respuesta


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

En ausencia de restricciones la complejidad sería 5^30, es decir el orden es de complejidad es exponencial.
El problema es de tipo NP complejo.

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Para preparar los modelos de datos y las funciones de costo iniciar por un modelo de búsqueda aleatoria, que si bien no ofrecerá una solucion óptima es el mas sencillo de implementar. Abajo está de hecho el código para este modelo, donde con 10000 iteraciones se encontro una solución de 33 salarios diarios para los dobles.

La búsqueda local, puede ser una buena opción, para tener mas intensificación que diversificación


In [ ]:
import csv
import random
import numpy as np
from math import ceil


In [ ]:
#Variables transversales, constantes y de entorno
global c_dias, c_actores, tomasxdia, tomas, arrmodelo, modeloini
c_actores,tomasxdia,tomas = 10,6,30
c_dias = ceil(tomas/tomasxdia)
#modelo básico de solucion donde se asigna un día de 1 a 5 a cada toma.
#se requiere esta lógica para permitir funcionamiento con programaciones de diferentes días
modeloini=[]
for i in range(1,c_dias+1):
  modeloini=modeloini+[i]*tomasxdia

arrmodelo = np.array(modeloini,dtype='uint8')

In [ ]:
#Leer archivo de programación de tomas: tiene columna de indice de tomas y no tiene header:
ruta="/content/Datos problema doblaje.csv"
lista=[]
with open(ruta) as fichero:
    data_reader = csv.reader(fichero, delimiter=',')
    for linea in data_reader:
        lista.append(linea)
tomas= np.array(lista, dtype='uint8')

In [ ]:
#Función para crear una solución aleatoria
def crear_solucion_rnd():
  sol_random = arrmodelo
  random.shuffle(sol_random)
  return sol_random

print(crear_solucion_rnd())

[1 5 4 2 2 2 4 1 5 4 1 5 3 3 3 2 5 2 2 1 4 5 4 1 3 4 3 1 3 5]


In [ ]:
#Función para calcular el costo de una solución
def costo(sol):  #parámetro es ndarray
  #cálculo de la matriz de programación: días en que trabaja cada actor
  matrizprog = np.zeros((30,11),dtype='uint8')
  for i in range(1,11):
   matrizprog[:,i-1] = np.multiply(sol,tomas[:,i])

  #cálculo de vector de dias por actor:
  dias_actor = np.zeros(10,dtype='uint8')
  for i in range(0,10):
    dias_actor[i] = len(np.unique(matrizprog[:,i]))-1
  return np.sum(dias_actor)

# sol = crear_solucion_rnd()
sol = np.array([1, 3, 3, 1, 2, 4, 5, 5, 2, 5, 1, 2, 4, 5, 3, 3, 1, 3, 5, 3, 2, 4, 4, 2, 1, 5, 4, 1, 4, 2],dtype='uint8')
costo(sol)


35

In [ ]:
#Búsqueda aleatoria
def busqueda_aleatoria():
  costo_min = 10000   #se inicializa alto para que se ajuste en la primera iteración

  for i in range(0,10000):
    sol_i = crear_solucion_rnd()
    costo_i = costo(sol_i)
    if costo_i <costo_min:
      costo_min = costo_i
      mejor_sol = sol_i
  return mejor_sol,costo_min

print(busqueda_aleatoria())


(array([2, 5, 2, 5, 4, 3, 2, 1, 5, 4, 3, 1, 4, 5, 5, 1, 2, 2, 4, 1, 5, 3,
       3, 1, 2, 4, 1, 4, 3, 3], dtype=uint8), 33)
